In [ ]:
# --- Blok 0: Inisialisasi, Fungsi Global, dan Pemuatan Data ---
import pandas as pd
import re
import string
import nltk
import ssl
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from afinn import Afinn
from textblob import TextBlob
from senticnet.senticnet import SenticNet # Pastikan senticnet terinstal
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os # Untuk memeriksa keberadaan file

# --- Konfigurasi dan Download Resource NLTK ---
def download_nltk_resources():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    resources = {
        "punkt": "tokenizers/punkt",
        "stopwords": "corpora/stopwords",
        "averaged_perceptron_tagger": "taggers/averaged_perceptron_tagger",
        "wordnet": "corpora/wordnet",
        "omw-1.4": "corpora/omw-1.4",
        "sentiwordnet": "corpora/sentiwordnet"
    }
    
    for resource_name, resource_path in resources.items():
        try:
            nltk.data.find(resource_path)
            print(f"[INFO] NLTK resource '{resource_name}' ({resource_path}) sudah ada.")
        except LookupError:
            print(f"[INFO] NLTK resource '{resource_name}' ({resource_path}) tidak ditemukan. Mengunduh...")
            try:
                nltk.download(resource_name, quiet=True)
                print(f"[INFO] Berhasil mengunduh '{resource_name}'.")
            except Exception as e:
                print(f"[ERROR] Gagal mengunduh '{resource_name}': {e}")
                print(f"         Coba jalankan secara manual: nltk.download('{resource_name}')")
        except Exception as e:
            print(f"[ERROR] Error saat memeriksa resource {resource_name}: {e}")

download_nltk_resources()
print("-" * 50)

# Fungsi untuk memberi label sentimen berdasarkan skor
def label_sentiment(score, pos_thresh=0.05, neg_thresh=-0.05):
    if score > pos_thresh:
        return "Positif"
    elif score < neg_thresh:
        return "Negatif"
    else:
        return "Netral"

# --- Pemuatan Data dan Sampling ---
csv_file_path_original = 'Data Pakai Label.csv'
csv_file_path_sample_translated = 'Data_Sample_Translated.csv' # File untuk menyimpan/memuat sampel terjemahan
translated_column_name = 'translated_content'
sample_size = 100
content_column_original = 'content' # Kolom teks asli Bahasa Indonesia

df_sampled_translated = None

# Coba muat sampel yang sudah ditranslasi jika ada
if os.path.exists(csv_file_path_sample_translated):
    try:
        df_sampled_translated = pd.read_csv(csv_file_path_sample_translated)
        if translated_column_name in df_sampled_translated.columns and len(df_sampled_translated) == sample_size:
            print(f"[INFO] Berhasil memuat sampel yang sudah ditranslasi dari: {csv_file_path_sample_translated}")
        else:
            print(f"[INFO] File sampel terjemahan ditemukan ({csv_file_path_sample_translated}) tetapi format/ukuran tidak sesuai. Akan dibuat ulang.")
            df_sampled_translated = None # Reset agar dibuat ulang
    except Exception as e:
        print(f"[PERINGATAN] Gagal memuat file sampel terjemahan ({csv_file_path_sample_translated}): {e}. Akan dibuat ulang.")
        df_sampled_translated = None

if df_sampled_translated is None:
    try:
        df_full = pd.read_csv(csv_file_path_original)
        print(f"[INFO] Berhasil memuat file asli: {csv_file_path_original}")
        print(f"Jumlah baris awal (keseluruhan): {len(df_full)}")

        if content_column_original not in df_full.columns:
            print(f"[ERROR] Kolom '{content_column_original}' tidak ditemukan di {csv_file_path_original}.")
            exit()

        # --- SAMPLING DATA ---
        if len(df_full) > sample_size:
            print(f"[INFO] Melakukan sampling sebanyak {sample_size} baris dari dataset asli.")
            df_to_process = df_full.sample(n=sample_size, random_state=42).reset_index(drop=True)
        else:
            print(f"[INFO] Ukuran dataset ({len(df_full)}) <= sample_size ({sample_size}). Menggunakan seluruh dataset.")
            df_to_process = df_full.copy()
        
        print(f"Jumlah baris untuk diproses/translasi: {len(df_to_process)}")

        # --- FASE 0 (OPSIONAL): Translasi Teks dari Bahasa Indonesia ke Bahasa Inggris ---
        # Jika Anda ingin melakukan translasi otomatis PADA SAMPEL, uncomment bagian di bawah.
        # Pastikan Anda sudah `pip install transformers sentencepiece sacremoses`
        
        from transformers import MarianMTModel, MarianTokenizer
        model_name_translation = "Helsinki-NLP/opus-mt-id-en"

        def translate_batch_hf(texts_series, model_name="Helsinki-NLP/opus-mt-id-en", batch_size=8):
            print(f"[INFO] Memuat model translasi: {model_name}")
            tokenizer = MarianTokenizer.from_pretrained(model_name)
            model = MarianMTModel.from_pretrained(model_name)
            print("[INFO] Model translasi dimuat.")
            
            texts = texts_series.tolist()
            translated_texts = []
            num_batches = (len(texts) + batch_size - 1) // batch_size
            
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = min((i + 1) * batch_size, len(texts))
                batch_texts_cleaned = [str(text) if pd.notnull(text) else "" for text in texts[start_idx:end_idx]]
                
                print(f"[INFO] Menerjemahkan batch {i+1}/{num_batches} (ukuran: {len(batch_texts_cleaned)})...")
                try:
                    inputs = tokenizer(batch_texts_cleaned, return_tensors="pt", padding=True, truncation=True, max_length=512)
                    translated_tokens = model.generate(**inputs)
                    batch_translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
                    translated_texts.extend(batch_translated)
                except Exception as e:
                    print(f"[ERROR] Gagal menerjemahkan batch {i+1}: {e}")
                    translated_texts.extend([""] * len(batch_texts_cleaned))
                time.sleep(0.1)
            return pd.Series(translated_texts, index=texts_series.index)

        # Lakukan translasi HANYA PADA SAMPEL
        print("[INFO] Memulai proses translasi untuk data sampel...")
        start_translate_time = time.time()
        df_to_process[translated_column_name] = translate_batch_hf(df_to_process[content_column_original])
        end_translate_time = time.time()
        print(f"[INFO] Proses translasi sampel selesai dalam {end_translate_time - start_translate_time:.2f} detik.")
        
        # *** BAGIAN INI YANG DITAMBAHKAN UNTUK KASUS TRANSLASI MANUAL ***
        # Jika Anda melakukan translasi manual, pastikan kolom hasil translasi sudah ada di `df_full`
        # dan skrip akan mengambilnya saat sampling.
        # Jika Anda uncomment blok translasi di atas, baris ini tidak akan dieksekusi
        # karena `translated_column_name` sudah dibuat.
        if translated_column_name not in df_to_process.columns:
            # Ini adalah placeholder jika translasi tidak dilakukan di skrip.
            # Idealnya, Anda sudah punya kolom ini di CSV asli jika translasi manual.
            print(f"[PERINGATAN] Kolom '{translated_column_name}' tidak ada. Akan dibuat placeholder dari '{content_column_original}'.")
            print("           Analisis mungkin tidak akurat. Pastikan Anda sudah menerjemahkan data Anda ke kolom ini.")
            df_to_process[translated_column_name] = df_to_process[content_column_original]
        # *** AKHIR BAGIAN TAMBAHAN ***

        df_sampled_translated = df_to_process.copy()

        # Simpan sampel yang sudah (atau dianggap sudah) ditranslasi
        try:
            df_sampled_translated.to_csv(csv_file_path_sample_translated, index=False, encoding='utf-8-sig')
            print(f"[INFO] Sampel yang (dianggap sudah) ditranslasi disimpan ke: {csv_file_path_sample_translated}")
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan file sampel terjemahan: {e}")

    except FileNotFoundError:
        print(f"[ERROR] File asli '{csv_file_path_original}' tidak ditemukan.")
        exit()
    except Exception as e:
        print(f"[ERROR] Terjadi kesalahan saat memuat atau sampling data: {e}")
        exit()

if df_sampled_translated is None:
    print(f"[ERROR] Gagal memuat atau membuat data sampel terjemahan. Harap periksa error sebelumnya.")
    exit()

df = df_sampled_translated.copy() # df sekarang adalah sampel yang sudah ada kolom terjemahannya
print(f"Jumlah baris data yang akan dianalisis: {len(df)}")
print("Contoh data yang akan dianalisis (2 baris pertama):")
print(df.head(2))
print("-" * 50)

[INFO] NLTK resource 'punkt' (tokenizers/punkt) sudah ada.
[INFO] NLTK resource 'stopwords' (corpora/stopwords) sudah ada.
[INFO] NLTK resource 'averaged_perceptron_tagger' (taggers/averaged_perceptron_tagger) sudah ada.
[INFO] NLTK resource 'wordnet' (corpora/wordnet) tidak ditemukan. Mengunduh...
[INFO] Berhasil mengunduh 'wordnet'.
[INFO] NLTK resource 'omw-1.4' (corpora/omw-1.4) tidak ditemukan. Mengunduh...
[INFO] Berhasil mengunduh 'omw-1.4'.
[INFO] NLTK resource 'sentiwordnet' (corpora/sentiwordnet) sudah ada.
--------------------------------------------------
[INFO] Berhasil memuat file: Data Pakai Label.csv
Jumlah baris awal: 10798
Kolom yang ada: ['userName', 'content', 'score', 'reviewCreatedVersion', 'at', 'Sentimen (Aurel)', 'Sentimen (Ade)', 'Sentimen (Eky)', 'Sentimen Akhir']
        userName                                            content  score  \
0  Made Darmawan  Saya kasi bintang 1 dulu ya. Tolong diperbaiki...      1   
1  windi tripani  Apknya memudahkan dalam 

/home/xerces/anaconda3/envs/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Memulai proses translasi...
[INFO] Memuat model translasi: Helsinki-NLP/opus-mt-id-en
[INFO] Model translasi dimuat.
[INFO] Menerjemahkan batch 1/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 2/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 3/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 4/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 5/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 6/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 7/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 8/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 9/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 10/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 11/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 12/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 13/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 14/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 15/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 16/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 17/1350 (ukuran: 8)...
[INFO] Menerjemahkan batch 18/1350 

KeyboardInterrupt: 

In [13]:
# --- Blok 1: Preprocessing Teks Bahasa Inggris ---
print("[FASE 1] Memulai preprocessing teks Bahasa Inggris...")
start_fase1_time = time.time()

# Fungsi-fungsi preprocessing
def to_lowercase(text):
    return text.lower() if isinstance(text, str) else ""

def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

def remove_mentions_hashtags(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    return text

def remove_punctuation(text):
    # Simpan tanda apostrof karena penting untuk kontraksi (misal, "don't")
    # punctuation_to_remove = string.punctuation.replace("'", "") 
    # return text.translate(str.maketrans('', '', punctuation_to_remove))
    # Untuk dictionary-based, seringkali lebih baik menghilangkan semua punctuation
    return text.translate(str.maketrans('', '', string.punctuation))


def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_extra_whitespace(text):
    return " ".join(text.split())

stop_words_en = set(nltk_stopwords.words('english'))
# Kata-kata negasi penting, jangan dihapus jika ada di kamus
negation_words = {"not", "no", "n't", "never", "don't", "can't", "isn't"}
stop_words_en = stop_words_en - negation_words

def remove_en_stopwords(tokens):
    return [word for word in tokens if word not in stop_words_en]

lemmatizer = WordNetLemmatizer()
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif nltk_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

def lemmatize_tokens_with_pos(tokens):
    nltk_tagged = nltk.pos_tag(tokens)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

# Pengecekan kolom hasil terjemahan
if translated_column_name not in df.columns:
    print(f"[ERROR] Kolom terjemahan '{translated_column_name}' tidak ada di DataFrame. Pastikan Fase 0 (Translasi) sudah benar.")
    exit()

# Menghapus baris jika teks terjemahan kosong
df.dropna(subset=[translated_column_name], inplace=True)
df.reset_index(drop=True, inplace=True)
print(f"Jumlah baris setelah menghapus NaN di '{translated_column_name}': {len(df)}")

# Pipeline Preprocessing
df['processed_text'] = df[translated_column_name].apply(to_lowercase)
df['processed_text'] = df['processed_text'].apply(remove_urls)
df['processed_text'] = df['processed_text'].apply(remove_mentions_hashtags)
df['textblob_ready_text'] = df['processed_text'].copy() # Untuk TextBlob, sebelum hapus tanda baca sepenuhnya

df['processed_text'] = df['processed_text'].apply(remove_punctuation)
df['processed_text'] = df['processed_text'].apply(remove_numbers)
df['processed_text'] = df['processed_text'].apply(remove_extra_whitespace)

df['tokens'] = df['processed_text'].apply(word_tokenize)
# STOPWORD REMOVAL DI-NONAKTIFKAN untuk dictionary based, bisa diaktifkan jika diperlukan
# df['tokens_no_stopwords'] = df['tokens'].apply(remove_en_stopwords)

df['lemmatized_tokens'] = df['tokens'].apply(lemmatize_tokens_with_pos)

end_fase1_time = time.time()
print(f"[FASE 1] Preprocessing teks selesai dalam {end_fase1_time - start_fase1_time:.2f} detik.")
print("\nContoh hasil preprocessing (5 baris pertama):")
print(df[[translated_column_name, 'processed_text', 'tokens', 'lemmatized_tokens']].head())
print("-" * 50)

[FASE 1] Memulai preprocessing teks Bahasa Inggris...
Jumlah baris setelah menghapus NaN di 'translated_content': 10798


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/xerces/nltk_data'
    - '/home/xerces/anaconda3/envs/venv/nltk_data'
    - '/home/xerces/anaconda3/envs/venv/share/nltk_data'
    - '/home/xerces/anaconda3/envs/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# --- Blok 2.1: Analisis Sentimen dengan AFINN ---
print("[FASE 2.1] Menerapkan analisis sentimen AFINN...")
start_fase2_1_time = time.time()

afinn_lexicon = Afinn()
def get_afinn_score(tokens):
    if not tokens: return 0
    # Afinn library bisa langsung menerima string, join token lebih baik
    return afinn_lexicon.score(" ".join(tokens))

df['afinn_score'] = df['tokens'].apply(get_afinn_score) # Menggunakan token asli (bukan lemmatized)
df['afinn_label'] = df['afinn_score'].apply(lambda x: label_sentiment(x, pos_thresh=0.1, neg_thresh=-0.1))

end_fase2_1_time = time.time()
print(f"[FASE 2.1] Analisis AFINN selesai dalam {end_fase2_1_time - start_fase2_1_time:.2f} detik.")
print("\nContoh hasil AFINN:")
print(df[[translated_column_name, 'afinn_score', 'afinn_label']].head())
print("-" * 50)

In [ ]:
# --- Blok 2.2: Analisis Sentimen dengan SentiWordNet ---
print("[FASE 2.2] Menerapkan analisis sentimen SentiWordNet...")
start_fase2_2_time = time.time()

def get_sentiwordnet_score(lemmatized_tokens_with_pos_tags):
    if not lemmatized_tokens_with_pos_tags:
        return 0.0, 0.0, 0.0

    sentiment_score_doc = 0.0
    pos_score_doc = 0.0
    neg_score_doc = 0.0
    
    # Kita sudah punya lemmatized_tokens, sekarang tinggal POS taggingnya
    # Jika lemmatized_tokens sudah punya POS tag, langsung gunakan.
    # Jika tidak, lakukan POS tagging pada lemmatized_tokens.
    # Untuk SentiWordNet, kata harus dilematisasi DENGAN POS tag yang benar.

    tagged_tokens = nltk.pos_tag(lemmatized_tokens_with_pos_tags) # POS tag pada token lemmatized

    for word, tag in tagged_tokens:
        wn_tag = nltk_tag_to_wordnet_tag(tag) # Fungsi ini sudah didefinisikan di Blok 1
        if wn_tag not in (nltk.corpus.wordnet.NOUN, nltk.corpus.wordnet.ADJ, 
                           nltk.corpus.wordnet.ADV, nltk.corpus.wordnet.VERB):
            continue

        # Lemmatize lagi dengan POS tag yang benar untuk SentiWordNet
        # (Meskipun sudah dilematisasi, terkadang bentuknya belum 100% dasar tanpa POS spesifik)
        lemma = lemmatizer.lemmatize(word, wn_tag) if wn_tag else word 
        
        synsets = list(swn.senti_synsets(lemma, wn_tag))
        if not synsets:
            # Jika tidak ada synset untuk lemma+tag, coba tanpa tag
            synsets = list(swn.senti_synsets(lemma))
            if not synsets:
                continue
        
        # Ambil skor dari synset pertama (paling umum)
        synset = synsets[0]
        pos_score = synset.pos_score()
        neg_score = synset.neg_score()
        
        pos_score_doc += pos_score
        neg_score_doc += neg_score
        sentiment_score_doc += (pos_score - neg_score)
        
    return pos_score_doc, neg_score_doc, sentiment_score_doc

swn_scores = df['lemmatized_tokens'].apply(get_sentiwordnet_score)
df['swn_pos_score'] = swn_scores.apply(lambda x: x[0])
df['swn_neg_score'] = swn_scores.apply(lambda x: x[1])
df['swn_sentiment_score'] = swn_scores.apply(lambda x: x[2])
df['swn_label'] = df['swn_sentiment_score'].apply(label_sentiment)

end_fase2_2_time = time.time()
print(f"[FASE 2.2] Analisis SentiWordNet selesai dalam {end_fase2_2_time - start_fase2_2_time:.2f} detik.")
print("\nContoh hasil SentiWordNet:")
print(df[[translated_column_name, 'swn_sentiment_score', 'swn_label']].head())
print("-" * 50)

In [ ]:
# --- Blok 2.3: Analisis Sentimen dengan SenticNet ---
print("[FASE 2.3] Menerapkan analisis sentimen SenticNet...")
start_fase2_3_time = time.time()

sn = SenticNet()
def get_senticnet_score(tokens_or_lemmas): # Bisa pakai tokens atau lemmatized_tokens
    if not tokens_or_lemmas: return 0.0
    
    score = 0.0
    count = 0
    for token in tokens_or_lemmas:
        try:
            # SenticNet bekerja dengan konsep, jadi lowercase
            concept_info = sn.concept(token.lower()) 
            polarity_value = float(concept_info['polarity_value'])
            score += polarity_value
            count += 1
        except KeyError: # Konsep tidak ditemukan di SenticNet
            continue
    return score / count if count > 0 else 0.0

# Coba dengan lemmatized_tokens karena SenticNet berbasis konsep
df['senticnet_score'] = df['lemmatized_tokens'].apply(get_senticnet_score) 
df['senticnet_label'] = df['senticnet_score'].apply(label_sentiment)

end_fase2_3_time = time.time()
print(f"[FASE 2.3] Analisis SenticNet selesai dalam {end_fase2_3_time - start_fase2_3_time:.2f} detik.")
print("\nContoh hasil SenticNet:")
print(df[[translated_column_name, 'senticnet_score', 'senticnet_label']].head())
print("-" * 50)

In [ ]:
# --- Blok 2.4: Analisis Sentimen dengan TextBlob ---
print("[FASE 2.4] Menerapkan analisis sentimen TextBlob...")
start_fase2_4_time = time.time()

def get_textblob_sentiment(text):
    if not isinstance(text, str) or not text.strip(): # Handle empty or non-string
        return 0.0, 0.0
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

# Gunakan 'textblob_ready_text' yang masih memiliki beberapa tanda baca
textblob_sentiments = df['textblob_ready_text'].apply(get_textblob_sentiment)
df['textblob_polarity'] = textblob_sentiments.apply(lambda x: x[0])
df['textblob_subjectivity'] = textblob_sentiments.apply(lambda x: x[1])
df['textblob_label'] = df['textblob_polarity'].apply(label_sentiment)

end_fase2_4_time = time.time()
print(f"[FASE 2.4] Analisis TextBlob selesai dalam {end_fase2_4_time - start_fase2_4_time:.2f} detik.")
print("\nContoh hasil TextBlob:")
print(df[[translated_column_name, 'textblob_polarity', 'textblob_label']].head())
print("-" * 50)

In [ ]:
# --- Blok 3: Tampilkan Hasil Gabungan dari Semua Kamus ---
print("[FASE 3] Menampilkan contoh hasil analisis gabungan (10 baris pertama):")

# Pastikan kolom Sentimen Akhir ada untuk perbandingan
ground_truth_col = 'Sentimen Akhir'
columns_to_display = ['content', translated_column_name]
if ground_truth_col in df.columns:
    columns_to_display.append(ground_truth_col)

columns_to_display.extend([
    'afinn_score', 'afinn_label', 
    'swn_sentiment_score', 'swn_label',
    'senticnet_score', 'senticnet_label',
    'textblob_polarity', 'textblob_label'
])

# Filter kolom yang benar-benar ada di DataFrame untuk menghindari error
columns_to_display = [col for col in columns_to_display if col in df.columns]

print(df[columns_to_display].head(10))
print("-" * 50)

In [ ]:
# --- Blok 4: Evaluasi dan Visualisasi ---
print("[FASE 4] Melakukan evaluasi dan visualisasi...")
start_fase4_time = time.time()

ground_truth_column = 'Sentimen Akhir' # Kolom sentimen manual Anda

def evaluate_sentiments_detailed(y_true, y_pred, method_name, labels_order=["Positif", "Netral", "Negatif"]):
    # Pastikan y_true dan y_pred memiliki panjang yang sama dan tidak kosong
    if len(y_true) == 0 or len(y_pred) == 0 or len(y_true) != len(y_pred):
        print(f"[PERINGATAN] Data tidak valid untuk evaluasi {method_name}. Panjang y_true: {len(y_true)}, y_pred: {len(y_pred)}")
        return None

    # Hapus baris NaN dari y_true dan y_pred secara bersamaan
    valid_indices = y_true.notna() & y_pred.notna()
    y_true_cleaned = y_true[valid_indices]
    y_pred_cleaned = y_pred[valid_indices]

    if len(y_true_cleaned) == 0:
        print(f"[PERINGATAN] Tidak ada data valid setelah membersihkan NaN untuk evaluasi {method_name}.")
        return None

    accuracy = accuracy_score(y_true_cleaned, y_pred_cleaned)
    # Dapatkan semua label unik yang ada di y_true_cleaned dan y_pred_cleaned
    unique_labels = sorted(list(set(y_true_cleaned.unique()) | set(y_pred_cleaned.unique())))
    
    # Filter labels_order agar hanya berisi label yang ada di data
    current_labels_order = [l for l in labels_order if l in unique_labels]
    if not current_labels_order: # Jika labels_order tidak relevan, gunakan unique_labels
        current_labels_order = unique_labels

    report = classification_report(y_true_cleaned, y_pred_cleaned, labels=current_labels_order, zero_division=0)
    cm = confusion_matrix(y_true_cleaned, y_pred_cleaned, labels=current_labels_order)

    print(f"\n--- Evaluasi untuk Metode: {method_name} ---")
    print(f"Akurasi: {accuracy:.4f}")
    print("Laporan Klasifikasi:")
    print(report)
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=current_labels_order, 
                yticklabels=current_labels_order)
    plt.title(f'Confusion Matrix - {method_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()
    
    return accuracy

if ground_truth_column in df.columns:
    print(f"\n[INFO] Melakukan evaluasi berdasarkan kolom '{ground_truth_column}'...")
    
    # Pastikan tidak ada NaN di kolom ground_truth_column sebelum evaluasi
    df_eval = df.copy() # Bekerja dengan salinan untuk evaluasi

    print(f"Jumlah baris sebelum menghapus NaN di '{ground_truth_column}': {len(df_eval)}")
    df_eval.dropna(subset=[ground_truth_column], inplace=True)
    print(f"Jumlah baris setelah menghapus NaN di '{ground_truth_column}': {len(df_eval)}")
    df_eval.reset_index(drop=True, inplace=True)

    if not df_eval.empty:
        evaluate_sentiments_detailed(df_eval[ground_truth_column], df_eval['afinn_label'], "AFINN")
        evaluate_sentiments_detailed(df_eval[ground_truth_column], df_eval['swn_label'], "SentiWordNet")
        evaluate_sentiments_detailed(df_eval[ground_truth_column], df_eval['senticnet_label'], "SenticNet")
        evaluate_sentiments_detailed(df_eval[ground_truth_column], df_eval['textblob_label'], "TextBlob")
    else:
        print(f"[PERINGATAN] Tidak ada data ground truth yang valid di kolom '{ground_truth_column}' untuk evaluasi setelah pembersihan NaN.")
else:
    print(f"[INFO] Kolom ground truth '{ground_truth_column}' tidak ditemukan. Evaluasi dilewati.")

end_fase4_time = time.time()
print(f"[FASE 4] Evaluasi dan visualisasi selesai dalam {end_fase4_time - start_fase4_time:.2f} detik.")
print("-" * 50)

In [ ]:
# --- Blok 5: Penyimpanan Hasil Akhir ---
print("[FASE 5] Menyimpan hasil akhir...")

output_csv_path = 'Data_Hasil_Sentiment_Dictionaries_Lengkap.csv'
try:
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig') # utf-8-sig untuk kompatibilitas Excel
    print(f"\n[INFO] Hasil lengkap disimpan ke: {output_csv_path}")
except Exception as e:
    print(f"[ERROR] Gagal menyimpan file CSV: {e}")

# Waktu total jika semua blok dijalankan berurutan dari awal
# total_execution_time = (end_fase1_time - start_fase1_time) + \
#                        (end_fase2_1_time - start_fase2_1_time) + \
#                        (end_fase2_2_time - start_fase2_2_time) + \
#                        (end_fase2_3_time - start_fase2_3_time) + \
#                        (end_fase2_4_time - start_fase2_4_time) + \
#                        (end_fase4_time - start_fase4_time) # Tambahkan waktu fase lain jika ada
# print(f"\n[INFO] Perkiraan total waktu eksekusi untuk Fase 1-4 (tanpa I/O data awal & translasi): {total_execution_time:.2f} detik.")
print("--- Proses Selesai ---")